## Red neuronal de tres capas con regularización

Los datos utilizados para ejercicio se pueden descargar de:
https://www.kaggle.com/datasets/hojjatk/mnist-dataset

Descarga los cuatro archivos.

Los archivos descomprimidos deberán de colocarse en la carpeta /mnist/

In [1]:
import numpy as np
import importlib
import math

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json, matplotlib
s = json.load( open("styles/bmh_matplotlibrc.json") )
matplotlib.rcParams.update(s)
from IPython.core.pylabtools import figsize
figsize(11, 5)
colores = ["#348ABD", "#A60628","#06A628"]

In [4]:
from ipywidgets import interact, interact_manual, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

In [ ]:
#from perceptron import RedMulticapa, makeX, makeY, logistica, derLogistica

# Datos

Los datos a utilizar son imágenes de dígitos.  En su formato original, se leen dos vectores:
* Los datos de **entrada** vienen en un vector 3D.
  * Cada renglón corresponde a un ejemplar de entrenamiento.
  * En cada renglón hay una matriz 2D con las intensidades de los pixels.
* Las etiquetas (dígito correcto que representan) vienen en un vector de una dimensión.

In [5]:
from mnist.read import read, printFull

filesDir = './mnist/'
trainingSetFile = filesDir + 'train-images-idx3-ubyte'
trainingSetLabelsFile = filesDir + 'train-labels-idx1-ubyte'
testSetFile = filesDir + 't10k-images-idx3-ubyte'
testSetLabelsFile = filesDir + 't10k-labels-idx1-ubyte'

###     /\ |__   __||  ____|| \ | | / ____||_   _|/ __ \ | \ | |
###    /  \   | |   | |__   |  \| || |       | | | |  | ||  \| |
###   / /\ \  | |   |  __|  | . ` || |       | | | |  | || . ` |
###  / ____ \ | |   | |____ | |\  || |____  _| |_| |__| || |\  |
### /_/    \_\|_|   |______||_| \_| \_____||_____|\____/ |_| \_|
### EN LAS SIQUIENTES DOS LINEAS SE LIMITA EL TAMAÑO DE LOS DATOS DE ENTRENAMIENTO,EN CASO
### DE INCREMENTAR LA CANTIDAD EL CALCULO TOMA DRASTICAMENTE MAS TIEMPO EN EL CASO DE APROXIMACION DEL GRADIENTE,SE INCLUYE
### IMPRESION DEL PROGRESO DEL METODO



trainData = read(fileName=trainingSetFile).astype(np.float64)
trainDataLabels = read(fileName=trainingSetLabelsFile).astype(np.float64)

testData = read(fileName=testSetFile).astype(np.float64)
testDataLabels = read(fileName=testSetLabelsFile).astype(np.float64)

Vector de  3  dimensiones:  (60000, 28, 28)  tipo  <class 'numpy.uint8'>
Vector de  1  dimensiones:  (60000,)  tipo  <class 'numpy.uint8'>
Vector de  3  dimensiones:  (10000, 28, 28)  tipo  <class 'numpy.uint8'>
Vector de  1  dimensiones:  (10000,)  tipo  <class 'numpy.uint8'>


In [6]:
import mnist.plot
from mnist.plot import muestraImagen

In [7]:
#Normalizamos los datos para que estén entre 0 y 1

trainData=trainData/255
testData=testData/255

In [8]:
## Función para ver como son los datos de MNIST

@interact(
    indice = (0, len(trainData) - 1)
)
def muestraImagenEntrenamiento(indice):
    muestraImagen(trainData, trainDataLabels, indice)

interactive(children=(IntSlider(value=29999, description='indice', max=59999), Output()), _dom_classes=('widge…

Para poder trabajar con la red neuronal, necesitaremos transformar esas entradas, de modo que los valores de las intensidades de los pixeles se encuentren en un solo renglón.  Las entradas a la red neuronal, deberán ser de la forma:
\begin{align}
  X &= \begin{bmatrix}
       x_1^{(1)} ... x_n^{(1)}  \\
       x_1^{(2)} ... x_n^{(2)}  \\
       ...\\
       x_1^{(m)} ... x_n^{(m)}
      \end{bmatrix}
\end{align}
También necesitaremos que las etiquetas formen una matriz donde la única columna distinta de cero, sea la correspondiente al dígito correcto:
\begin{align}
  Y &= \begin{bmatrix}
       0, ..., y_{label_0} = 1 , ... ,0 \\
       ... \\
       0, ..., y_{label_n} = 1 , ... ,0 \\
      \end{bmatrix}
\end{align}


In [10]:
## Define las matrices X y Y, como se muestra arriba
## A partir de trainData y trainDataLabels
## TIP: usar reshape
def makeX(datosEntrenamiento):
    return datosEntrenamiento.reshape((datosEntrenamiento.shape[0],-1))
    

def makeY(etiquetasEntrenamiento):
    return np.eye(10)[etiquetasEntrenamiento.astype(int)]
    

In [11]:
X = makeX(trainData)
print("X shape=",X.shape)

Y = makeY(trainDataLabels)
print("Y shape=",Y.shape)

X shape= (60000, 784)
Y shape= (60000, 10)


In [12]:
## Repite lo mismo con los datos de validación
XTest = makeX(testData)
print("XTest shape=",XTest.shape)

YTest = makeY(testDataLabels)
print("YTest shape=",YTest.shape)

XTest shape= (10000, 784)
YTest shape= (10000, 10)


## Red con tres capas
La red neuronal que se utilizará es una red neuronal de tres capas:
* Entrada
* Oculta
* Salida
La forma genérica de la red se ilustra a continuación.  Sólo que la red de este ejercicio tendrá más neuronas en cada capa.

<img src="figuras/Red3Capas.png"/>

Para este ejercicio el número de neuronas será:
* 784 + 1 (28x28 pixeles más la unidad de sesgo)
* 25 + 1 unidades en la capa oculta
* 10 neuronas de salida (una por cada dígito)
Por lo tanto, las dimensiones de las matrices de pesos son:
* $\Theta^{(0)} \rightarrow (25 \times 785)$
* $\Theta^{(1)} \rightarrow (10 \times 26)$

## Regularización
Se buscará mantener los pesos pequeños para evitar los debordes y el sobreajuste a los datos
de entrenamiento.  Para ello, la función $J$, además de medir el error en la predicción, será
incrementada cuando los pesos incrementen su magnitud.
\begin{align}
  J(\Theta) =& - \frac{1}{m} \left[ \sum_{i=1}^m \sum_{k=1}^K   y_k^{(i)} \log(h_\Theta(x^{(i)}))_k  +
            (1 - y_k^{(i)}) \log(1 - h_\Theta(x^{(i)}))_k   \right]    +  \frac{\lambda}{2m} \sum_{l=1}^{L-1} \sum_{i=1}^{s_L} \sum_{j=1}^{s_{l+1}} (\theta_{ji}^{(l)})^2
\end{align}
De este modo, al calcular el gradiente se hace la siguiente modificación:
\begin{align}
 \nabla^{(l)} =& \frac{1}{m}\Delta^{(l)} \\
 \nabla^{(l)}[:,1:] =& \nabla^{(l)}[:,1:] + \frac{\lambda}{m} \Theta^{(l)}[:,1:]
\end{align}

In [13]:
# Esta red tiene
print(25*785 + 10*26, ' conexiones.')

19885  conexiones.


In [14]:
def logistica(val):
    return 1/(1+np.exp(-val))

def derLogistica(val):
    return logistica(val) * (1 - logistica(val))

In [15]:
## Programa una clase RedNeuronal con la arquitectura anterior y que tome en cuenta la regularización.

## Debe tener:
## - como *atributos* las matrices de pesos Theta_0 y Theta_1
## - un *constructor* que reciba como parámetro opcional otra red y copie sus pesos,
##   si no recibe nada los inicializa aleatoriamente.
## - método para asignar valores aleatorios a estas matrices
## - método para devolver todos los pesos en una sola matriz columna.
## - metodo para reconstruir matrices del tamaño de las matrices de pesos, a partir del vector.
## Define una función de entrenamiento para la red.

# Cuando agregues el término de la regularización, recuerda hacer los cambios
# únicamente sobre las componentes que no corresponden al bias.
# Puedes utilizar notación como:
# M[:,1:] = f(M[:,1:])
# donde f(x) es una función cualquiera que depende de x.

class RedMulticapa:
    def __init__(self,otra=None):
        if otra:
            self.Theta0 = otra.Theta0
            self.Theta1 = otra.Theta1
        else:
            self.Theta0 = np.random.rand(25,785)
            self.Theta1 = np.random.rand(10,26)
         
    
    def pesos_a_vector(self):
        return np.concatenate((self.Theta0.flatten(),self.Theta1.flatten()))
        
        
    
    def reconstruct_matrices(matriz):
        return matriz[:25*785].reshape((25,785)), matriz[25*785:].reshape((10,26))
    
    def matrizDeConfusion(self, Y):
        num_clases = Y.shape[1]
        num_muestras = Y.shape[0]
        _, _, _, _, Y_pred = self.feed_forward(X)  # Salida de la red para las muestras X
        Y_true_classes = np.argmax(Y, axis=1)
        Y_pred_classes = np.argmax(Y_pred, axis=1)

        confusion_matrix = np.zeros((num_clases, num_clases), dtype=int)

        for i in range(num_clases):
            for j in range(num_clases):
                confusion_matrix[i, j] = np.sum((Y_true_classes == i) & (Y_pred_classes == j))

        return confusion_matrix

    # setattr(RedMulticapa,'matrizDeConfusion',matrizDeConfusion)

In [16]:
## Programa el método *feedforward(self, X, vector=None)*
## Cuando reciba vector, usará estos pesos, en lugar de los propios.
##
def feed_forward(self, X, vector=None):
    if vector is not None:
        Theta0, Theta1 = RedMulticapa.reconstruct_matrices(vector)
    else:
        Theta0 = self.Theta0
        Theta1 = self.Theta1
    a1 = np.insert(X, 0, 1, axis=1)
    z2 = a1 @ Theta0.T
    a2 = np.insert(logistica(z2), 0, 1, axis=1)
    z3 = a2 @ Theta1.T
    a3 = logistica(z3)

    # Guardar las activaciones como atributos de la instancia
    self.A0 = a1
    self.A1 = a2
    self.A2 = a3

    return a1, z2, a2, z3, a3
    
setattr(RedMulticapa,'feed_forward',feed_forward)

In [17]:
importlib.reload(mnist.plot)

from mnist.plot import muestraActividad

In [18]:
RedA = RedMulticapa()
RedA.feed_forward(XTest)

(array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[35.13897431, 38.13792891, 35.08753695, ..., 31.68049478,
         35.61410407, 35.5566171 ],
        [60.84485379, 54.62300125, 58.03083358, ..., 58.261158  ,
         58.78957044, 54.2311604 ],
        [20.68218896, 22.08193193, 22.79597127, ..., 19.72384298,
         20.29402581, 18.16202329],
        ...,
        [56.01913384, 60.003893  , 59.21554555, ..., 53.0010806 ,
         59.51163193, 58.21378925],
        [52.23539761, 51.75934361, 56.91085248, ..., 48.49031039,
         52.67802667, 51.45547539],
        [79.86271589, 81.50154865, 86.31527277, ..., 82.93858643,
         87.29522527, 80.40009113]]),
 array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
         1.        ],
        [1.        , 1.     

In [19]:
# Esto sólo es ilustrativo, por eso usamos el conjunto de prueba,
# que es más pequeño

@interact(index = (0, len(XTest) - 1))
def muestraActividad0(index):
    muestraActividad(RedA, index)

interactive(children=(IntSlider(value=4999, description='index', max=9999), Output()), _dom_classes=('widget-i…

In [20]:
## Programa un método para calcular el error.
## Usar la entropía cruzada.
## Recuerda dar la opción de enviar un vector de pesos modificados.
def calc_error(self, xtest, ytest, pesos, lambdaR):
    a1, z2, a2, z3, a3 = self.feed_forward(xtest, pesos)

    m = len(ytest)
    return -1 / m * np.sum(ytest * np.log(a3) + (1 - ytest) * np.log(1 - a3)) + lambdaR / (2 * m) * (np.sum(self.Theta0[:, 1:] ** 2) + np.sum(self.Theta1[:, 1:] ** 2))

setattr(RedMulticapa,'calc_error',calc_error)

In [22]:
RedB = RedMulticapa(RedA)
RedB.calc_error(XTest,YTest,RedB.pesos_a_vector(),1)

121.0271255437028

In [23]:
## Programa el método *aproximaGradiente(self, X, Y, lambdaR = 0.0)*
## Esta función perturbará los pesos uno a uno, por una cantidad
## epsilon = 0.0004 y devolverá un vector con el gradiente.
def approx_gradient(self, X, Y):
    epsilon = 0.0004
    pesos = self.pesos_a_vector()
    gradient = np.zeros(pesos.shape)
    for i in range(len(pesos)):
        pesos[i] += epsilon
        error_mas = self.calc_error(X,Y,pesos,1)
        pesos[i] -= 2*epsilon
        error_menos = self.calc_error(X,Y,pesos,1)
        gradient[i] = (error_mas - error_menos)/(2*epsilon)
        pesos[i] += epsilon
    return gradient

setattr(RedMulticapa,'approx_gradient',approx_gradient)
aprox = RedB.approx_gradient(X[0:1,:], YTest[0:1,:])

In [24]:
## Programa el algoritmo de propagación hacia atrás
## de modo que reciba X, Y y lambda=0.0
## Guardará como atributos los valores calculados
## para el error y los gradientes de las matrices de pesos
def back_propagate(self,x,y,lambdaR=0.0):
    a1, z2, a2, z3, a3 = self.feed_forward(x)
    delta3 = a3 - y
    delta2 = (delta3 @ self.Theta1) * (a2 * (1 - a2))
    delta2 = delta2[:, 1:]
    m = len(y)
    self.error = -1 / m * np.sum(y * np.log(a3) + (1 - y) * np.log(1 - a3)) + lambdaR / (2 * m) * (np.sum(self.Theta0[:, 1:] ** 2) + np.sum(self.Theta1[:, 1:] ** 2))
    self.grad0 = delta2.T @ a1 / m + lambdaR / m * np.insert(self.Theta0[:, 1:], 0, 0, axis=1)
    self.grad1 = delta3.T @ a2 / m + lambdaR / m * np.insert(self.Theta1[:, 1:], 0, 0, axis=1)


setattr(RedMulticapa,'back_propagate',back_propagate)

In [25]:
## Agrega ahora el método para realizar descenso por el gradiente
def gradient_descent(self,X,Y,alfa,ciclos,lambdaR=0.0):
    for i in range(ciclos):
        self.back_propagate(X,Y,lambdaR)
        self.Theta0 -= alfa * self.grad0
        self.Theta1 -= alfa * self.grad1
    
setattr(RedMulticapa,'gradient_descent',gradient_descent)

In [26]:
# Tendrás que correr el entrenamiento varias veces para que el sistema funcione.
# Toma en cuenta que X es una matriz bastante grande, por lo que será un poco tardado.
# Observa que aparecerán advertencias de que el cálculo de la exponencial en la
# función de evaluación está desbordando.
# No olvides revisar tus metádatos, prueba entrenamiento con diferentes lambdaR. 

@interact_manual()
def entrenaRedC():
    RedB.gradient_descent(X, Y, alfa=0.1, ciclos = 10, lambdaR=1.0)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

Antes de correr lo siguiente, se debe correr lo celda anterior y su boton "Run Interact"

In [27]:
# Si ya funciona bien tu entrenamiento, corre algunos ciclos más para reducir el error
# más significativamente.

@interact(index = (0, len(X) - 1))
def muestraActividad0(index):
    muestraActividad(RedB, index)

interactive(children=(IntSlider(value=29999, description='index', max=59999), Output()), _dom_classes=('widget…

## Matriz de confusión
Calcula los valores de la matriz de confusión para evaluar el desempeño de tu red después de haberla entrenado.

<table>
<thead>
<tr><th colspan="2"></th><th colspan="2">Clase predicha</th></tr>
<tr><th colspan="2"></th><th>1</th><th>0</th></tr>
</thead>
<tr><th rowspan="2">Clase correcta</th><th>1</th><td>TP</td><td>FN</td></tr>
<tr><th>0</th><td>FP</td><td>TN</td></tr>
</table>

Donde:
* **TP**: Verdaderos positvos, valores que se evaluaron como verdaderos y eran verdaderos.
* **FP**: Falsos positivos, valores que se evaluaron como verdaderos pero eran errados. 
* **FN**: Falsos negativos, valores evaluados como falsos pero debian ser verdaderos.
* **TN**: Verdaderos negativos, valores evaluados como falsos y eran falsos.

En nuestro ejercicio debemos ver en los valores resultantes, para cada ejemplar tenemos 10 elementos de salida, 
si en el vector de salida se activa la posición correspondiente a la etiqueta. Por ejemplo, si la salida es de la forma $[0,1,0,0,0,0,0,0,0,0]$ y la etiqueta es un 1 entonces tienes un verdadero positivo por el 1 y 9 verdaderos negativos por todos los que no se activaron.

In [28]:
## Inserta aquí el código para calcular la matriz de confusión
'''
# Movemos lo presente a la clase RedMulticapa para evitar problemas de compatibilidad
def matrizDeConfusion(self, Y):
    num_clases = Y.shape[1]
    num_muestras = Y.shape[0]
    _, _, _, _, Y_pred = self.feed_forward(X)  # Salida de la red para las muestras X
    Y_true_classes = np.argmax(Y, axis=1)
    Y_pred_classes = np.argmax(Y_pred, axis=1)

    confusion_matrix = np.zeros((num_clases, num_clases), dtype=int)

    for i in range(num_clases):
        for j in range(num_clases):
            confusion_matrix[i, j] = np.sum((Y_true_classes == i) & (Y_pred_classes == j))

    return confusion_matrix

setattr(RedMulticapa,'confusion',matrizDeConfusion)
'''

"\n# Movemos lo presente a la clase RedMulticapa para evitar problemas de compatibilidad\ndef matrizDeConfusion(self, Y):\n    num_clases = Y.shape[1]\n    num_muestras = Y.shape[0]\n    _, _, _, _, Y_pred = self.feed_forward(X)  # Salida de la red para las muestras X\n    Y_true_classes = np.argmax(Y, axis=1)\n    Y_pred_classes = np.argmax(Y_pred, axis=1)\n\n    confusion_matrix = np.zeros((num_clases, num_clases), dtype=int)\n\n    for i in range(num_clases):\n        for j in range(num_clases):\n            confusion_matrix[i, j] = np.sum((Y_true_classes == i) & (Y_pred_classes == j))\n\n    return confusion_matrix\n\nsetattr(RedMulticapa,'confusion',matrizDeConfusion)\n"

In [29]:
RedB.matrizDeConfusion(Y)

array([[   0, 5923,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 6742,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5958,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 6131,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5842,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5421,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5918,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 6265,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5851,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 5949,    0,    0,    0,    0,    0,    0,    0,    0]])

In [30]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()